In [ ]:
import torch
from torch.utils.data import DataLoader
from data.dataset import MonitorDetectionDataset
import torchvision.transforms as transforms
import random
import numpy as np
from models.yolov3quad import 

In [ ]:
# CONFIG
CSV_PATH = './labels.csv'
IMG_PATH = './images/'
FOLD = 0
SEED = 123
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
def seed_all():
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(SEED)
        torch.cuda.manual_seed_all(SEED)
        torch.backends.cudnn.benchmark = True

In [ ]:
# LOAD DATA
transform = transforms.Compose([
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.Resize((256, 256)), 
    transforms.ToTensor(),
    ])
train_dataset = MonitorDetectionDataset(IMG_PATH, CSV_PATH, transform, 
    train=True, fold=FOLD, seed=SEED)
test_dataset = MonitorDetectionDataset(IMG_PATH, CSV_PATH, transform,
    train=False, fold=FOLD, seed=SEED)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)